# Exploring and Clustering the Neighborhoods in Toronto

This notebook is part of the capstone project for the [IBM Data Science Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-science) course.
In this project I’m exploring and clustering the neighborhoods in Toronto.

## Preparing Data

In [1]:
import requests
import lxml.html as html
import pandas as pd

Downloading and parsing the wiki page:

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki_url)
wiki_doc = html.fromstring(wiki_page.content)

Finding the table element.
NOTE: Changes to the page will require changes in this code.

In [3]:
wiki_table = wiki_doc.xpath('//*[@id="mw-content-text"]/div/table[1]')

# Let's make sure we found the right table
import re

re_postal_code = r'\nM\d[A-Z]\n'
re_postal_code_flags = re.MULTILINE | re.UNICODE
if len(wiki_table) == 0 or re.search(re_postal_code, wiki_table[0].text_content(), re_postal_code_flags) == None:
    raise Exception('Could not find the table of postal codes. Consider updating the XPath.')

postal_codes_table = wiki_table[0]
postal_codes_table

<Element table at 0x22727646138>

Getting location data from the given CSV document. I tried using geocoder, but it takes forever.

In [4]:
df_locations = pd.read_csv('https://cocl.us/Geospatial_data')
df_locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Creating the dataframe. I’m creating a dictionary first and then I’m creating a dataframe out of it. Location data is also added right away.

In [5]:
rows = postal_codes_table.findall('.//tr')
if len(rows) == 0:
    raise Exception('Could not find any rows in the table')

EXPECTED_COLS_NUM = 3
table_dict = { 'PostalCode': [], 'Borough': [], 'Neighborhood': [], 'Latitude': [], 'Longitude': [] }

for row in rows:
    cols = row.findall('.//td')
    num_cols = len(cols)

    # Skip rows without td elements (like the header)
    if num_cols == 0:
        continue

    # Make sure we always the expected number of columns
    if num_cols != EXPECTED_COLS_NUM:
        raise Exception('Expected exactly {} columns but got {}.'.format(EXPECTED_COLS_NUM, num_cols))

    borough = cols[1].text_content().strip()

    # Ignore rows without borough as per the task description
    if borough == 'Not assigned':
        continue

    neighborhoods = cols[2].text_content().strip().split(' / ')

    # Make neighborhood same as borough if the former isn't specified
    if len(neighborhoods) == 0 or neighborhoods[0] == 'Not assigned':
        neighborhoods = [borough]

    postal_code = cols[0].text_content().strip()
    table_dict['PostalCode'].append(postal_code)
    table_dict['Borough'].append(borough)
    table_dict['Neighborhood'].append(', '.join(neighborhoods))

    # Getting location data
    loc = df_locations.loc[df_locations['Postal Code'] == postal_code]
    table_dict['Latitude'].append(loc['Latitude'].values[0])
    table_dict['Longitude'].append(loc['Longitude'].values[0])

df = pd.DataFrame.from_dict(table_dict)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Rendering the Map

Installing required dependencies for drawing maps.

In [6]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.



In [7]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.



Importing packages required for maps

In [44]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# map rendering library
import folium

Getting the location of Toronto

In [45]:
city_address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(city_address)
city_latitude = location.latitude
city_longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(city_latitude, city_longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Creating a map of Toronto with neighborhoods superimposed on top

In [46]:
toronto_map = folium.Map(location=[city_latitude, city_longitude], zoom_start=10)

# Adding markers to the map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

## Exploring & Clustering

### Exploring a subset of neighborhoods in Toronto

In [ ]:
I’m gonna work with boroughs that contain the word `Toronto` only, as was suggested in the task description. This will limit the data to explore.

In [47]:
toronto_neighborhoods = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [51]:
limited_toronto_map = folium.Map(location=[city_latitude, city_longitude], zoom_start=12)

# Adding markers to the map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(limited_toronto_map)  
    
limited_toronto_map

Now I’m gonna start utilizing the Foursquare API to explore the neighborhoods and segment them.

I’ll be using Foursquare API. Since this notebook is public, I’m gonna store the credentials locally in a separate file.

In [53]:
from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read('secrets.cfg')

foursquare_client_id = parser.get('secrets', 'foursquare_client_id')
foursquare_client_secret = parser.get('secrets', 'foursquare_client_secret')
foursquare_version = '20180605'

Next I’m gonna explore venues in each neighborhood

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            foursquare_client_id, 
            foursquare_client_secret, 
            foursquare_version, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighborhood'],
                                 latitudes=toronto_neighborhoods['Latitude'],
                                 longitudes=toronto_neighborhoods['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst
 Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

New dataframe shape and preview:

In [55]:
print(toronto_venues.shape)
toronto_venues.head()

(1692, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let’s get the total number of venues:

In [56]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst\n Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,77,77,77,77,77,77
Christie,17,17,17,17,17,17
Church and Wellesley,83,83,83,83,83,83
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


Let's find out how many unique categories can be curated from all the returned venues

In [57]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


### Exploring each neighborhood

In [59]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [60]:
toronto_onehot.shape

(1692, 233)

#### Next, I’m gonna group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [61]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Business reply mail Processing CentrE,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.0625,0.125,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.012987,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012987,...,0.000000,0.00000,0.00,0.012987,0.000000,0.000000,0.012987,0.000000,0.000000,0.00
5,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.024096,0.012048,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012048,...,0.000000,0.00000,0.00,0.000000,0.000000,0.012048,0.000000,0.012048,0.012048,0.00
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


#### Let's confirm the new size

In [62]:
toronto_grouped.shape

(39, 233)

#### Let's print each neighborhood along with the top 5 most common venues

In [63]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print('----' + hood + '----')
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.04
4          Bakery  0.04


----Business reply mail Processing CentrE----
                  venue  freq
0                Garden  0.06
1         Auto Workshop  0.06
2  Fast Food Restaurant  0.06
3         Burrito Place  0.06
4                   Spa  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst
 Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1  Airport Terminal  0.12
2             Plane  0.06
3     Boat or Ferry  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.0

#### Let's put that into a *pandas* dataframe

First, let’s write a function to sort the venues in descending order.

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let’s create the new dataframe and display the top 10 venues for each neighborhood.

In [101]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Restaurant,Farmers Market,Café,Cheese Shop,Bakery,Cosmetics Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,Business reply mail Processing CentrE,Light Rail Station,Fast Food Restaurant,Auto Workshop,Brewery,Spa,Burrito Place,Recording Studio,Pizza Place,Garden,Gym / Fitness Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Bar,Rental Car Location,Sculpture Garden,Plane,Coffee Shop,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Thai Restaurant,Burger Joint,Ice Cream Shop,Café,Middle Eastern Restaurant,Spa


### Clustering

Next I’m gonna cluster the neighborhoods into 10 different clusters

In [102]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([8, 8, 8, 8, 2, 0, 8, 2, 8, 7])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [103]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neighborhoods
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Pub,Park,Theater,Mexican Restaurant,Breakfast Spot,Restaurant,Bakery,Café,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Yoga Studio,Creperie,Boutique,Burger Joint,Burrito Place,Restaurant,Café,College Auditorium,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,8,Clothing Store,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Cosmetics Shop,Thai Restaurant,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,8,Coffee Shop,Café,Italian Restaurant,Restaurant,American Restaurant,Cosmetics Shop,Bakery,Beer Bar,Diner,Cocktail Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,9,Health Food Store,Trail,Pub,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Finally, let's visualize the resulting clusters

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters